In [1]:
import pandas as pd
import os
import time
import csv
import re
import codecs
import requests
from bs4 import BeautifulSoup
    
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException, ElementClickInterceptedException

In [ ]:
print(selenium.__version__)

In [2]:
def text_file_to_dataframe(filename):
    # Check if the text file exists
    if os.path.exists(filename):
        
        # Read the text file and create a list of lines
        with open(filename, "r") as file:
            lines = file.readlines()
            content = [line.strip() for line in lines]

        # Create a DataFrame from the list
        df = pd.DataFrame(content, columns=['links'])
        
        return df
    else:
        print(f"{filename} does not exist.")
        return None

# Loading Links & Creating DataFrame

In [3]:
input_filename = "final_links.txt"
df = text_file_to_dataframe(input_filename)

# Sub-Setting With The last 2,905 Links in Shawn's List

In [4]:
df = df[-2905:]

In [5]:
df.shape

(2905, 1)

# Scraper Code

In [6]:
# Define a function to extract the publisher element text
def get_publisher_text():
    try:
        # Find the element with the EditionDetails class
        time.sleep(10)
        edition_details_element = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CLASS_NAME, 'EditionDetails')))
        
        # Find the nested element with the data-testid attribute
        publisher_element = edition_details_element.find_elements_by_css_selector('div[data-testid="contentContainer"]')
        
        return publisher_element[1].text

    except NoSuchElementException:
        return ""
    except TimeoutException:
        print("Timeout: EditionDetails element not found within the specified time")
        return ""
    except ElementClickInterceptedException:
        return ""

In [7]:
# S C R A P I N G   B R O W S E R
driver = webdriver.Chrome()

reviews_file_exists = os.path.isfile("reviews.csv")
print("Does Review CSV File Exists?")
print(reviews_file_exists)
print("\n")

index_file_exists = os.path.isfile("index.txt")

# checking last value of index in order to continue from last position
if index_file_exists:
    with open("index.txt", "r") as index_file:
        last_index = int(index_file.read())
else:
    last_index = 0

print("Output of last_index")
print(last_index)

# iterate over the dataframe
for index, row in df.iloc[last_index:].iterrows():
    print("Ouput of index")
    print(index)
    
    print("Ouput of row")
    print(row)

    #============================================================================================================
    # L O A D    P A G E    O N    S C R A P I N G    B R O W S E R
    #============================================================================================================
    driver.get(row['links'])
    try:
        print("Waiting for Title Element to Load....")
        title_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'BookPageTitleSection__title')))
        print("Assigning Title")
        title = title_element.text
    except TimeoutException:
        print(f"Timed out while waiting for title for book {row['links']}, moving on to next book")
        continue
    
    print("Ouput of title")
    print(title)
    
    #============================================================================================================
    # P U B L I S H E R   D A T A
    #============================================================================================================
    # First, try to find and click the button
    button_found = False
    try:
        time.sleep(10)
        button = driver.find_element_by_css_selector('button[aria-label="Book details and editions"]')
        button.click()
        button_found = True
        # Wait for the publisher element to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "EditionDetails")))
    except (NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException):
        print("Button not found or not clickable")

    # If the button is not found or not clickable, the get_publisher_text function will still run
    publisher = get_publisher_text()

    # Print the extracted publisher text
    if publisher is not None:
        print("=====================")
        print(" P U B L I S H E R ")
        print("=====================")
        print(publisher)
        print("=====================")
    else:
        print("Publisher text not found")


    #============================================================================================================
    # D E F I N I T I O N    S E C T I O N
    #============================================================================================================
    try:
        print("Waiting for Synopsis and Reviews to load....")
        time.sleep(20)
        synopsis_and_review_list = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'Formatted')))
        print(f"The variable length of book {index+1} = ")
        print(len(synopsis_and_review_list))
        synopsis_length = len(synopsis_and_review_list)
        print("Output of synopsis_length")
        print(synopsis_length)
    except TimeoutException:
        print(f"Timed out while waiting for synopsis and review list for book {title}, moving on to next book")
        continue
    
    try:
        page_count_element = driver.find_element_by_css_selector('p[data-testid="pagesFormat"]')
        page_count = page_count_element.text
        
        print("Ouput of page_count")
        print(page_count)
    except NoSuchElementException:
        page_count = ""
    
    try:
        year_element = driver.find_element_by_css_selector('p[data-testid="publicationInfo"]')
        year = year_element.text
        
        print("Ouput of year")
        print(year)
    except NoSuchElementException:
        year = ""
    
    try:
        review_count_element = driver.find_element_by_css_selector('span[data-testid="reviewsCount"]')
        review_count = review_count_element.text
        
        print("Ouput of review_count")
        print(review_count)
        
        # converting string to interger with RegEx
        review_count_match = re.search(r'\d+', review_count)
        if review_count_match:
            review_count = int(review_count_match.group())
            print("Ouput of RegEx: review_count")
            print(review_count)
        else:
            review_count = 0
            print("Ouput of No_RegEx: review_count")
            print(review_count)
            
    except NoSuchElementException:
        review_count = 0
        print("Ouput of No_Count: review_count")
        print(review_count)
    
    try:
        rating_count_element = driver.find_element_by_css_selector('span[data-testid="ratingsCount"]')
        rating_count = rating_count_element.text
        print("Ouput of rating_count")
        print(rating_count)
    except NoSuchElementException:
        rating_count = ""
    
    try:
        rating_element = driver.find_element_by_class_name('RatingStatistics__rating')    
        rating = rating_element.text
        print("Ouput of rating")
        print(rating)
        
    except NoSuchElementException:
        rating = ""
    
    try:
        genre_class_element = driver.find_element_by_class_name('BookPageMetadataSection__genres')
        genres_text_element = genre_class_element.find_elements_by_css_selector('.BookPageMetadataSection__genreButton .Button__labelItem')
        genre = genres_text_element[0].text
        print("Ouput of genre")
        print(genre)
        
    except NoSuchElementException:
        genre = ""
    
    try:
        author_element = driver.find_element_by_class_name('ContributorLink__name')
        author = author_element.text
        print("Ouput of author")
        print(author)

    except NoSuchElementException:
        author = ""
        
    book_reviews = [title, page_count, year, review_count, rating, rating_count, genre, author,publisher]
    
    # current row in DataFrame. create these columns
    # and add book information to current dataframe row
    df.at[index, 'page_count'] = page_count
    df.at[index, 'publisher'] = publisher
    df.at[index, 'year'] = year
    df.at[index, 'review_count'] = review_count
    df.at[index, 'rating'] = rating
    df.at[index, 'rating_count'] = rating_count
    df.at[index, 'genre'] = genre
    df.at[index, 'author'] = author

    
    # =====================
    # T E S T   P R I N T
    # =====================
    print(f"Book {index}:")
    print(title)
    print("==============")
    print(page_count)
    print("\n")
    print("==============")
    print(year)
    print("\n")
    print("==============")
    print("Total Reviews: ")
    print(review_count)
    print("\n")
    print("==============")
    print(rating)
    print("\n")
    print("==============")
    print(rating_count)
    print("\n")
    print("==============")
    print(genre)
    print("\n")
    print("==============")
    print(author)
    print("\n")
    print("==============")
    print(publisher)
    print("\n")
    # ==================================================================
    # R E V I E W S
    # ==================================================================
    # iterating 5 times for reviews offset by 3 
    # list of elements "synopsis_and_review_list" has synopsis
    # and reviews
    review_index = 3
    for i in range(5):
        if review_index < synopsis_length:
            print(f"Review {i + 1}")
            print("==============")
            print(synopsis_and_review_list[review_index].text)

            # saving current review to temporary list
            book_reviews.append(synopsis_and_review_list[review_index].text)
            review_index += 1
        else:
            book_reviews.append("")
    # ==================================================================
    # S A V I N G   P L A C E   A F T E R   E A C H   I T E R A T I O N
    # ==================================================================
    with open("reviews.csv", "a", newline='', encoding='utf-8') as reviews_file:
        csv_writer = csv.writer(reviews_file)
        if not reviews_file_exists:
            csv_writer.writerow(['Book Name', 'page_count', 'year', 'review_count', 'rating', 'rating_count', 'genre', 'author', 'publisher', 'Review 1', 'Review 2', 'Review 3', 'Review 4', 'Review 5'])
            reviews_file_exists = True
        csv_writer.writerow(book_reviews)

    # ==================================================================
    # S A V I N G   P L A C E   A F T E R   E A C H   I T E R A T I O N
    # ==================================================================
    # save place of index to file for resuming later
    with open("index.txt", "w") as index_file:
        index_file.write(str(index + 1))

# D O N E    W I T H    S C R A P E R
driver.quit()

if not reviews_file_exists:
    open("reviews.csv", "a", newline='', encoding='utf-8').close()

if not index_file_exists:
    open("index.txt", "a").close()

Does Review CSV File Exists?
True


Output of last_index
2001
Ouput of index
4001
Ouput of row
links    https://www.goodreads.com/book/show/17725997-t...
Name: 4001, dtype: object
Waiting for Title Element to Load....
Assigning Title
Ouput of title
The Renaldis #1
Tempted by Her Boss
 P U B L I S H E R 
July 3, 2013 by Samhain Publishing, Ltd.
Waiting for Synopsis and Reviews to load....
The variable length of book 4002 = 
33
Output of synopsis_length
33
Ouput of page_count
202 pages, ebook
Ouput of year
First published July 2, 2013
Ouput of review_count
148 reviews
Ouput of RegEx: review_count
148
Ouput of rating_count
2,636 ratings
Ouput of rating
3.81
Ouput of genre
Romance
Ouput of author
Karen Erickson
Book 4001:
The Renaldis #1
Tempted by Her Boss
202 pages, ebook


First published July 2, 2013


Total Reviews: 
148


3.81


2,636 ratings


Romance


Karen Erickson


July 3, 2013 by Samhain Publishing, Ltd.


Review 1
I really enjoyed Karen Erickson's Trilogy of Merry widows hist

Waiting for Title Element to Load....
Assigning Title
Ouput of title
Lucky Paws Petsitting Mystery #1
Death by Chocolate Lab
 P U B L I S H E R 
February 28, 2017 by Kensington
Waiting for Synopsis and Reviews to load....
The variable length of book 4004 = 
33
Output of synopsis_length
33
Ouput of page_count
343 pages, Paperback
Ouput of year
First published February 28, 2017
Ouput of review_count
277 reviews
Ouput of RegEx: review_count
277
Ouput of rating_count
1,092 ratings
Ouput of rating
3.79
Ouput of genre
Mystery
Ouput of author
Bethany Blake
Book 4003:
Lucky Paws Petsitting Mystery #1
Death by Chocolate Lab
343 pages, Paperback


First published February 28, 2017


Total Reviews: 
277


3.79


1,092 ratings


Mystery


Bethany Blake


February 28, 2017 by Kensington


Review 1
A doggone good start to a new cozy series! Sisters, Daphne (a free-spirited pet sitter), and Piper (a veterinarian), live in the quaint, lakeside village of Sylvan Creek, PA, in a cozy farmhouse belonging

Waiting for Title Element to Load....
Assigning Title
Ouput of title
Brotherhood of Blood #3
Taken by the Night
 P U B L I S H E R 
October 30, 2007 by Avon
Waiting for Synopsis and Reviews to load....
The variable length of book 4005 = 
33
Output of synopsis_length
33
Ouput of page_count
384 pages, Paperback
Ouput of year
First published October 1, 2007
Ouput of review_count
75 reviews
Ouput of RegEx: review_count
75
Ouput of rating_count
2,011 ratings
Ouput of rating
4.05
Ouput of genre
Paranormal Romance
Ouput of author
Kathryn Smith
Book 4004:
Brotherhood of Blood #3
Taken by the Night
384 pages, Paperback


First published October 1, 2007


Total Reviews: 
75


4.05


2,011 ratings


Paranormal Romance


Kathryn Smith


October 30, 2007 by Avon


Review 1
5+ stars – Vampire/Historical Romance

This is my favorite book in the extraordinary Brotherhood of the Blood vampire historical romance series. The series has a unique take on vampire mythology, centering on six soldiers who bec

Waiting for Title Element to Load....
Assigning Title
Ouput of title
Brotherhood of Blood #3
Taken by the Night
 P U B L I S H E R 
October 30, 2007 by Avon
Waiting for Synopsis and Reviews to load....
The variable length of book 4006 = 
33
Output of synopsis_length
33
Ouput of page_count
384 pages, Paperback
Ouput of year
First published October 1, 2007
Ouput of review_count
75 reviews
Ouput of RegEx: review_count
75
Ouput of rating_count
2,011 ratings
Ouput of rating
4.05
Ouput of genre
Paranormal Romance
Ouput of author
Kathryn Smith
Book 4005:
Brotherhood of Blood #3
Taken by the Night
384 pages, Paperback


First published October 1, 2007


Total Reviews: 
75


4.05


2,011 ratings


Paranormal Romance


Kathryn Smith


October 30, 2007 by Avon


Review 1
5+ stars – Vampire/Historical Romance

This is my favorite book in the extraordinary Brotherhood of the Blood vampire historical romance series. The series has a unique take on vampire mythology, centering on six soldiers who bec

Waiting for Title Element to Load....
Assigning Title
Ouput of title
Wedded by Scandal #3
How to Marry a Marquess
 P U B L I S H E R 
August 14, 2017 by Entangled: Scandalous
Waiting for Synopsis and Reviews to load....
The variable length of book 4007 = 
33
Output of synopsis_length
33
Ouput of page_count
279 pages, ebook
Ouput of year
First published August 14, 2017
Ouput of review_count
313 reviews
Ouput of RegEx: review_count
313
Ouput of rating_count
2,820 ratings
Ouput of rating
3.91
Ouput of genre
Historical Romance
Ouput of author
Stacy Reid
Book 4006:
Wedded by Scandal #3
How to Marry a Marquess
279 pages, ebook


First published August 14, 2017


Total Reviews: 
313


3.91


2,820 ratings


Historical Romance


Stacy Reid


August 14, 2017 by Entangled: Scandalous


Review 1
4 friends-to-lovers angsty stars

I love this series, and I loved this couple - I was looking forward to their story, and while I definitely enjoyed it, there was more angst than I felt necessary. There a

...

Waiting for Title Element to Load....
Assigning Title
Ouput of title
Hazard and Somerset #4
Guilt by Association
 P U B L I S H E R 
June 8, 2018 by Hodgkin and Blount
Waiting for Synopsis and Reviews to load....
The variable length of book 4330 = 
33
Output of synopsis_length
33
Ouput of page_count
390 pages, Kindle Edition
Ouput of year
First published June 8, 2018
Ouput of review_count
289 reviews
Ouput of RegEx: review_count
289
Ouput of rating_count
1,996 ratings
Ouput of rating
4.27
Ouput of genre
Mystery
Ouput of author
Gregory Ashe
Book 4329:
Hazard and Somerset #4
Guilt by Association
390 pages, Kindle Edition


First published June 8, 2018


Total Reviews: 
289


4.27


1,996 ratings


Mystery


Gregory Ashe


June 8, 2018 by Hodgkin and Blount


Review 1
Reread/listen July 2021

GAAAAHH! I love these two fools.



I was ready to defend Nico but again he is just a little twat. His insecurities turned him into a whiny brat. Ugh! Anyhoo Hazard would do that to you. Only Somers 

Waiting for Title Element to Load....
Assigning Title
Ouput of title
Montana Adventures #2
Savage: Two lives linked across time by blood and destiny...
 P U B L I S H E R 
February 27, 2018
Waiting for Synopsis and Reviews to load....
The variable length of book 4331 = 
33
Output of synopsis_length
33
Ouput of page_count
306 pages, Kindle Edition
Ouput of year
Published February 27, 2018
Ouput of review_count
48 reviews
Ouput of RegEx: review_count
48
Ouput of rating_count
1,347 ratings
Ouput of rating
4.57
Ouput of author
D.W. Ulsterman
Book 4330:
Montana Adventures #2
Savage: Two lives linked across time by blood and destiny...
306 pages, Kindle Edition


Published February 27, 2018


Total Reviews: 
48


4.57


1,347 ratings





D.W. Ulsterman


February 27, 2018


Review 1
Excellent novel.

First book I read written by this author was The Writer, first book in San Juan Islands Mystery series. This book like the others were written, will make you smell the dirt or shiver in the sno

Waiting for Title Element to Load....
Assigning Title
Ouput of title
Stranded #2
Trial by Fire
 P U B L I S H E R 
June 13, 2013 by Puffin Books
Waiting for Synopsis and Reviews to load....
The variable length of book 4333 = 
33
Output of synopsis_length
33
Ouput of page_count
208 pages, Paperback
Ouput of year
First published June 13, 2013
Ouput of review_count
196 reviews
Ouput of RegEx: review_count
196
Ouput of rating_count
1,507 ratings
Ouput of rating
4.07
Ouput of genre
Adventure
Ouput of author
Jeff Probst
Book 4332:
Stranded #2
Trial by Fire
208 pages, Paperback


First published June 13, 2013


Total Reviews: 
196


4.07


1,507 ratings


Adventure


Jeff Probst


June 13, 2013 by Puffin Books


Review 1
The sequel to Stranded. Everything is Starting over for the stranded kids. Finding Water,Food and dealing with wild animals. Can they survive?
Review 2
OMGGG I love this book so much:)!!!!!!!!! When my friend and were reading some reviews on another website, we were shouting 

Waiting for Title Element to Load....
Assigning Title
Ouput of title
Cooking for Wizards, Warriors and Dragons: 125 Unofficial Recipes Inspired By The Witcher, Game of Thrones, The Broken Earth and Other Fantasy Favorites
 P U B L I S H E R 
September 21, 2021 by Media Lab Books
Waiting for Synopsis and Reviews to load....


KeyboardInterrupt: 